Загрузка и импорт нужных функций и пакетов, загрузка репозитория и переход в рабочую папку (персональные данные скрыты, названия файлов не настоящие)

In [ ]:
!pip install fasttext
import fasttext, os, zipfile, re, nltk, random
nltk.download('punkt')
from nltk.corpus import PlaintextCorpusReader
from nltk.util import ngrams
from collections import Counter
from scipy import spatial,stats
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from sklearn import metrics
from scipy.linalg import norm

!git config --global user.email "почта"
!git config --global user.name "юзернейм"
!git clone ссылка на репозиторий --depth=1

folder=zipfile.ZipFile('путь к архиву с текстами', 'r')
folder.extractall('/content')

folder_with_texts1='путь к папке, куда распаковались тексты'
spisok=eval(open('список авторов','r',encoding='utf-8').read())
table=[x.strip().split('\t') for x in open("таблица с метоинформацией текстов",'r',encoding='cp1251').readlines()]
model = fasttext.load_model('путь к модели')

Все нужные функции

In [ ]:
def str_prepr(stri):
  return re.sub(' {2,}',' ',re.sub('[^\w\s-]', '', re.sub('_','',re.sub('--','―',stri)))).strip()

def date_list(dates,fileids):
  for d in [fileid[:-4].split('_')[-1] for fileid in fileids]: 
    if len(d)==4:
      dates.append(d)
  dates=np.unique(np.array([dates]))
  return list(dates)

def dates_dict(date_dict,dates,fileids):
    for date in dates:
      date_list=[]
      for fileid in fileids:
        date_id=fileid[:-4].split('_')[-1]
        if str(date)==date_id:
          date_list.append(fileid)  
      date_dict[date]=date_list 
    return date_dict

def text_for_learning(text_of_text,date_dict,dates):
  for date in dates:
    xml_of_date=corpus_text.raw(fileids=[*date_dict[date]])
    xmls=[xml+'</html>' for xml in xml_of_date.split('</html>')]
    for xml in xmls:
      soup = BeautifulSoup(xml, 'lxml')
      text_of_text=text_of_text+'\n\n'.join([str_prepr(x.get_text()) for x in soup('p') if len(x.contents)!=0 and x.contents[0].name!='noindex']).lower()+'\n\n'
  final_text=re.sub('\n{4,}','\n\n',text_of_text)
  return final_text.split()


def get_data(gen):
    try:
        for elem in gen:
            yield elem
    except (RuntimeError, StopIteration):
        return  

def generate_ngrams_content(date_dict,dates,ngrams_content):
  for date in dates:
    xml_of_date=corpus_text.raw(fileids=[*date_dict[date]])
    xmls=[xml+'</html>' for xml in xml_of_date.split('</html>')]
    date_ngram_list=[]
    for xml in xmls:
      soup = BeautifulSoup(xml, 'lxml')
      text_of_text='\n\n'.join([x.get_text() for x in soup('p') if len(x.contents)!=0 and x.contents[0].name!='noindex']).lower()
      text_of_text = re.sub('̀', '', text_of_text)
      text_of_text = re.sub('…','...',text_of_text)
      text_of_text = re.sub('(\. +?){4,}|(\. +?){2}','', text_of_text)
      text_of_text = re.sub('--','―',text_of_text)
      ngrams_from_text=get_data(ngrams(text_of_text,4))
      for gram in ngrams_from_text:
        new_gram=''.join([g for g in gram])
        date_ngram_list.append(new_gram)   
    ngrams_content.append(date_ngram_list)
  ngrams_content=np.array(ngrams_content)
  return ngrams_content

def metric_chunks_content(date_dict,dates,metric_chunks_all):
  for date in dates:
    xml_of_date=corpus_text.raw(fileids=[*date_dict[date]])
    xmls=[xml+'</html>' for xml in xml_of_date.split('</html>')]
    date_chunks=[]
    for xml in xmls:
      soup = BeautifulSoup(xml, 'lxml')
      text_of_text='\n\n'.join([x.get_text() for x in soup('p') if len(x.contents)!=0 and x.contents[0].name!='noindex']).lower()
      for line in text_of_text.split('\n'):
        if len(re.findall(' \.',line))==0 and len(line)>1: #цепочки нулей-единиц
          chain = ''
          for char in line:
              if char in 'аяоеуюыёиэ':
                  chain = chain + '0'
              elif char in '֎̀':
                  chain = chain[:-1] + '1'
          if '1' in chain and '0' in chain:
            date_chunks.append(chain)
    metric_chunks_all.append(date_chunks)
  metric_chunks_all=np.array(metric_chunks_all)
  return metric_chunks_all

def find_max(year_vectors,date_dict,dates,ngrams_content,metric_content):
    k=1
    dlina=len(date_dict.keys())
    dist_list=[]
    raz_names=[]
    maxes=[]
    while k!=dlina:
      print(k)
      period1_ngrams=np.concatenate(ngrams_content[0:k])
      period2_ngrams=np.concatenate(ngrams_content[k:dlina])
      joined_list_ngrams=np.unique(np.concatenate((period1_ngrams,period2_ngrams), axis=0))
      frec_first_ngrams=Counter(period1_ngrams) #частотный словарь первого периода
      frec_second_ngrams=Counter(period2_ngrams)
      k1=len(frec_first_ngrams)*1000 #коэффициент, чтобы привести частоты к ipm
      k2=len(frec_second_ngrams)*1000 #то же самое для второго
      vector1_ngrams=np.array([frec_first_ngrams[word]/k1 for word in joined_list_ngrams]) #частотный вектор первого корпуса
      vector2_ngrams=np.array([frec_second_ngrams[word]/k2 for word in joined_list_ngrams])

      period1_metric=np.concatenate(metric_content[0:k])
      period2_metric=np.concatenate(metric_content[k:dlina])
      joined_list_metric=np.unique(np.concatenate((period1_metric,period2_metric), axis=0))
      frec_first_metric=Counter(period1_metric) #частотный словарь первого периода
      frec_second_metric=Counter(period2_metric)
      k1=len(frec_first_metric)*1000 #коэффициент, чтобы привести частоты к ipm
      k2=len(frec_second_metric)*1000 #то же самое для второго
      vector1_metric=np.array([frec_first_metric[word]/k1 for word in joined_list_metric][:100])  #частотный вектор первого корпуса
      vector2_metric=np.array([frec_second_metric[word]/k2 for word in joined_list_metric][:100])

      embedd1=np.mean(np.array([year_vectors[y] for y in dates[0:k]]), axis=0 )
      embedd2=np.mean(np.array([year_vectors[y] for y in dates[k:dlina]]), axis=0 )

      vector1=stats.zscore(np.concatenate((vector1_metric,vector1_ngrams,embedd1)))
      print(np.std(vector1))
      vector2=stats.zscore(np.concatenate((vector2_metric,vector2_ngrams,embedd2)))
      print(np.std(vector2))

      period1_start=dates[0] #число-начало первого 
      period1_end=dates[k-1] #число-конец первого периода 
      period2_start=dates[k] #число-начало второго периода
      period2_end=dates[dlina-1] #число-конец второго периода
      if  period1_start!=period1_end and period2_start!=period2_end: 
        razbienie_name='('+ period1_start+'_'+period1_end+')'+'___'+'('+period2_start+'_'+period2_end+')'
      elif  period1_start!=period1_end and period2_start==period2_end:
        razbienie_name='('+ period1_start+'_'+period1_end+')'+'___'+'('+period2_start+')'
      else:
        razbienie_name='('+ period1_start+')'+'___'+'('+period2_start+'_'+period2_end+')'
          #Создаем список разбиений с id файлов получившихся периодов
      raz_names.append(razbienie_name)
      cos_dist=1 - np.dot(vector1,vector2)/(norm(vector1)*(norm(vector2)))
      dist_list.append([razbienie_name,cos_dist])
      print(razbienie_name,cos_dist)
      k=k+1
    for i in range(1,len(dist_list)-1):
      if dist_list[i][1]>dist_list[i-1][1] and dist_list[i][1]>dist_list[i+1][1]:
        maxes.append([name]+dist_list[i])
    return maxes 

Делим на 2 периода и находим локальные максимумы расстояния

In [ ]:
maxes_for_write=[]
for name in spisok:
    print(name)
    corpus_text = PlaintextCorpusReader(folder_with_texts1+'/'+name, '.*')
    fileids=sorted(corpus_text.fileids(), key=lambda fileid: int(fileid[:-4].split('_')[-1][:4]))
    dates=date_list([],fileids)
    date_dict=dates_dict({},dates,fileids)
    year_vectors={}
    for year in dates:
      year_vectors[year]=np.mean(np.array([model.get_word_vector(w) for w in text_for_learning('',date_dict,[year])]), axis=0 )
    metric_content=metric_chunks_content(date_dict,dates,[])
    ngrams_content=generate_ngrams_content(date_dict,dates,[])
    maxes_for_write=maxes_for_write+find_max(year_vectors,date_dict,dates,ngrams_content,metric_content)

Проверяем все локальные максимумы

In [ ]:
results=[]
for r in maxes_for_write:
      row1=r[0]
      row2=r[1]

      period1_board=int(row1.split('_')[1])
      period2_board=int(row2.split('_')[0])

      corpus_root = 'путь к корпусу'+'имя нужного автора'
      corpus_text = PlaintextCorpusReader(corpus_root, '.*')
      fileids=sorted([f for f in corpus_text.fileids()], key=lambda fileid: int(fileid[:-4].split('_')[-1][:4]))
      date_dict={}
      dates=[]
      for d in [fileid[:-4].split('_')[-1] for fileid in fileids]:
          if len(d)==4:
            dates.append(d)
      dates=sorted(list(set(dates)))
      period1=[d for d in dates if int(d[:4])<=int(period1_board) and int(d[-4:])<=int(period1_board)]
      period2=[d for d in dates if int(d[:4])>=int(period2_board)]

      def shuffle_files(per,fileids):
              needed_datemarks=[]
              needed_fileids=[]
              dates1=per
              for date in sorted(dates1):
                for fileid in fileids:
                  date_id=fileid[:-4].split('_')[-1]
                  if date_id==str(date):
                    needed_datemarks.append(fileid.split('_')[-1][:-4])
                    needed_fileids.append(fileid)
              random.shuffle(needed_datemarks)
              new_fileids=list(zip(needed_fileids,needed_datemarks))
              return new_fileids

      count=0
      for i in range(0,100):
            print(i)
            shuffeled_fileids1=sorted([f for f in shuffle_files(period1,fileids)+shuffle_files(period2,fileids)], key=lambda fileid: int(fileid[1]))
            print(shuffeled_fileids1)
            date_dict=dates_dict({},dates,shuffeled_fileids1)
            year_vectors={}
            for year in dates:
              year_vectors[year]=np.mean(np.array([model.get_word_vector(w) for w in text_for_learning('',date_dict,[year])]), axis=0 )
            metric_content=metric_chunks_content(date_dict,dates,[])
            ngrams_content=generate_ngrams_content(date_dict,dates,[])
            m1=find_max(year_vectors,date_dict,dates,ngrams_content,metric_content)

            if m1 == '('+row1+')___('+row2+')':
              count+=1
              print('count',count)
              results.append((r,count))

Делим на 3 периода и находим локальные максимумы расстояния (приведем только новые функции)

In [ ]:
def new_d(year_vectors,p1,p2,fileids):
      date_dict1=dates_dict({},p1,fileids)
      date_dict2=dates_dict({},p2,fileids)

      period1_ngrams=np.concatenate(generate_ngrams_content(date_dict1,p1,[]))
      period2_ngrams=np.concatenate(generate_ngrams_content(date_dict2,p2,[]))
      joined_list_ngrams=np.unique(np.concatenate((period1_ngrams,period2_ngrams), axis=0))
      frec_first_ngrams=Counter(period1_ngrams) #частотный словарь первого периода
      frec_second_ngrams=Counter(period2_ngrams)
      k1=len(frec_first_ngrams)*1000000 #коэффициент, чтобы привести частоты к ipm
      k2=len(frec_second_ngrams)*1000000 #то же самое для второго
      vector1_ngrams=[frec_first_ngrams[word]/k1 for word in joined_list_ngrams]  #частотный вектор первого корпуса
      vector2_ngrams=[frec_second_ngrams[word]/k2 for word in joined_list_ngrams]

      period1_metric=np.concatenate(metric_chunks_content(date_dict1,p1,[]))
      period2_metric=np.concatenate(metric_chunks_content(date_dict2,p2,[]))
      joined_list_metric=np.unique(np.concatenate((period1_metric,period2_metric), axis=0))
      frec_first_metric=Counter(period1_metric) #частотный словарь первого периода
      frec_second_metric=Counter(period2_metric)
      k1=len(frec_first_metric)*1000000 #коэффициент, чтобы привести частоты к ipm
      k2=len(frec_second_metric)*1000000 #то же самое для второго
      vector1_metric=[frec_first_metric[word]/k1 for word in joined_list_metric]  #частотный вектор первого корпуса
      vector2_metric=[frec_second_metric[word]/k2 for word in joined_list_metric]

      embedd1=np.mean(np.array([year_vectors[y] for y in p1]), axis=0 )
      embedd2=np.mean(np.array([year_vectors[y] for y in p2]), axis=0 )

      vector1=stats.zscore(np.concatenate((vector1_metric,vector1_ngrams,embedd1)))
      vector2=stats.zscore(np.concatenate((vector2_metric,vector2_ngrams,embedd2)))

      period1_start=p1[0] #число-начало первого 
      period1_end=p1[-1] #число-конец первого периода 
      period2_start=p2[0]
      period2_end=p2[-1] #число-конец второго периода
      if  period1_start!=period1_end and period2_start!=period2_end: 
        razbienie_name='('+ period1_start+'_'+period1_end+')'+'___'+'('+period2_start+'_'+period2_end+')'
      elif  period1_start!=period1_end and period2_start==period2_end:
        razbienie_name='('+ period1_start+'_'+period1_end+')'+'___'+'('+period2_start+')'
      else:
        razbienie_name='('+ period1_start+')'+'___'+'('+period2_start+'_'+period2_end+')'
      return [razbienie_name,spatial.distance.cosine(vector1, vector2)]

def three_subs(subs,a):
    for i in range(0,len(a)):
      for k in range(1,len(a)-1):
        for j in range(len(a),1,-1):
          period1=a[:i+1]
          period2=a[i+1:k+1]
          period3=a[k+1:j+1]
          if len(period1)+len(period2)+len(period3)==len(a) and len(period1)>0 and len(period2)>0 and len(period3)>0 and [period1,period2,period3] not in subs:
            subs.append([period1,period2,period3])
    return subs

for name in spisok: 
    print(name)
    corpus_root = folder_with_texts1+'/'+name
    corpus_text = PlaintextCorpusReader(corpus_root, '.*')
    fileids=sorted(corpus_text.fileids(), key=lambda fileid: int(fileid[:-4].split('_')[-1][:4]))
    dates=list(date_list([],fileids))
    date_dict=dates_dict({},dates,fileids)
    year_vectors={}
    for year in dates:
      year_vectors[year]=np.mean(np.array([model.get_word_vector(w) for w in text_for_learning('',date_dict,[year])]), axis=0 )
    subs=list(three_subs([],dates))
    print(len(subs))
    q=0
    d4=[]
    for sub in subs:
        q=q+1
        print(q)
        d1=new_d(year_vectors,sub[0],sub[1],fileids)
        d2=new_d(year_vectors,sub[1],sub[2],fileids)
        dx=d1+d2
        d4.append(dx)
    for i in range(1,len(d4)-1):
      if d4[i][1]>d4[i-1][1] and d4[i][1]>d4[i+1][1] and d4[i][3]>d4[i-1][3] and d4[i][3]>d4[i+1][3] and len(d4[i][0]+d4[i][2])==50:
        print('разбиение на три части',d4[i])

Вот как получаются все варианты разбиения на 3 части:

In [ ]:
three_subs([],[1900,1901,1902,1903,1904,1905,1906,1907,1908])

[[[1900], [1901], [1902, 1903, 1904, 1905, 1906, 1907, 1908]],
 [[1900], [1901, 1902], [1903, 1904, 1905, 1906, 1907, 1908]],
 [[1900], [1901, 1902, 1903], [1904, 1905, 1906, 1907, 1908]],
 [[1900], [1901, 1902, 1903, 1904], [1905, 1906, 1907, 1908]],
 [[1900], [1901, 1902, 1903, 1904, 1905], [1906, 1907, 1908]],
 [[1900], [1901, 1902, 1903, 1904, 1905, 1906], [1907, 1908]],
 [[1900], [1901, 1902, 1903, 1904, 1905, 1906, 1907], [1908]],
 [[1900, 1901], [1902], [1903, 1904, 1905, 1906, 1907, 1908]],
 [[1900, 1901], [1902, 1903], [1904, 1905, 1906, 1907, 1908]],
 [[1900, 1901], [1902, 1903, 1904], [1905, 1906, 1907, 1908]],
 [[1900, 1901], [1902, 1903, 1904, 1905], [1906, 1907, 1908]],
 [[1900, 1901], [1902, 1903, 1904, 1905, 1906], [1907, 1908]],
 [[1900, 1901], [1902, 1903, 1904, 1905, 1906, 1907], [1908]],
 [[1900, 1901, 1902], [1903], [1904, 1905, 1906, 1907, 1908]],
 [[1900, 1901, 1902], [1903, 1904], [1905, 1906, 1907, 1908]],
 [[1900, 1901, 1902], [1903, 1904, 1905], [1906, 1907, 

Проверяем варианты разбиения на 3 периода

In [ ]:
results=[]
for r in maxes_for_write:

    period1_board=int(r[0].split('_')[1])
    period2_board1=int(r[1].split('_')[0])
    period2_board2=int(r[1].split('_')[1])
    period3_board=int(r[2].split('_')[0])

    corpus_root = 'путь к папке'+'имя нужного автора'
    corpus_text = PlaintextCorpusReader(corpus_root, '.*')
    fileids=sorted([f for f in corpus_text.fileids()], key=lambda fileid: int(fileid[:-4].split('_')[-1][:4]))
    date_dict={}
    dates=[]
    for d in [fileid[:-4].split('_')[-1] for fileid in fileids]:
        if len(d)==4:
          dates.append(d)
    dates=sorted(list(set(dates)))
    period1=[d for d in dates if int(d[:4])<=int(period1_board)]
    period2=[d for d in dates if int(d[:4])>=int(period2_board1) and int(d[-4:])<=int(period2_board2)]
    period3=[d for d in dates if int(d[:4])>=int(period3_board)]

    dates1=period1+period2
    print(dates1)
    dates2=period2+period3
    print(dates2)

    count=0
    for i in range(0,100):
      print(i)
      shuffeled_fileids1=sorted([f for f in shuffle_files(period1,fileids)+shuffle_files(period2,fileids)], key=lambda fileid: int(fileid[1]))
      print(shuffeled_fileids1)
      date_dict=dates_dict({},dates1,shuffeled_fileids1)
      year_vectors={}
      for year in dates1:
        year_vectors[year]=np.mean(np.array([model.get_word_vector(w) for w in text_for_learning('',date_dict,[year])]), axis=0 )
      metric_content=metric_chunks_content(date_dict,dates1,[])
      ngrams_content=generate_ngrams_content(date_dict,dates1,[])
      m1=find_max(year_vectors,date_dict,dates1,ngrams_content,metric_content)

      shuffeled_fileids2=sorted([f for f in shuffle_files(period2,fileids)+shuffle_files(period3,fileids)], key=lambda fileid: int(fileid[1]))
      print(shuffeled_fileids2)
      date_dict=dates_dict({},dates2,shuffeled_fileids2)
      year_vectors={}
      for year in dates2:
        year_vectors[year]=np.mean(np.array([model.get_word_vector(w) for w in text_for_learning('',date_dict,[year])]), axis=0 )
      metric_content=metric_chunks_content(date_dict,dates2,[])
      ngrams_content=generate_ngrams_content(date_dict,dates2,[])
      m2=find_max(year_vectors,date_dict,dates2,ngrams_content,metric_content)
      if m1 == '('+r[0]+')___('+r[1]+')' and m2 == '('+r[1]+')___('+r[2]+')':
        count+=1
        print('count',count)
      results.append([m1,m2,count])